In [1]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from io import StringIO
from selenium.webdriver.chrome.options import Options
import pandas as pd
from datetime import datetime
import os
from PyPDF2 import PdfReader
import re
import glob 
from itertools import repeat

In [2]:
start_time = datetime.now()

In [3]:
dir = 'Y:\delete\ASEAN'
for f in os.listdir(dir):
    os.remove(os.path.join(dir, f))

In [4]:
ser = Service("C:/Users/AIMPAI0/Downloads/chromedriver.exe")
op = webdriver.ChromeOptions()
prefs = {"download.default_directory" : "Y:\\delete\\ASEAN","plugins.always_open_pdf_externally": True}
op.add_experimental_option("prefs",prefs)
driver = webdriver.Chrome(service=ser, options=op)
driver.maximize_window()  
driver.get("https://www.hsa.gov.sg/cosmetic-products/asean-cosmetic-directive")
driver.implicitly_wait(5)
driver.find_element(by=By.XPATH, value='//*[@id="main_TB6F372E3007_Col01"]/div/ol/li[2]/a').click()
time.sleep(10)
driver.close()

In [5]:
path = 'Y:\\delete\\ASEAN'
filenames = glob.glob(path + "\*.pdf")
for f in filenames:
    reader = PdfReader(f)

In [18]:
reader = PdfReader("Y:/delete/ASEAN/*.pdf")
number_of_pages = len(reader.pages)
number_of_pagescount = 0
text = ""
count = 0
Register_number = []
annex = []
while count < number_of_pages:
    page = reader.pages[count]
    text = ""
    text = page.extract_text()
    text = text.replace("\n", "")
    count +=1
    if text[0:9].strip() in ('ANNEX III' , 'Annex II'):
        if re.search("[0-9]+-.?[0-9]+-.?[0-9]+", text):
            d = re.findall("[0-9]+-.?[0-9]+-.?[0-9]+", text)
            Register_number.extend(d)
            annex.extend(repeat(text[0:9],len(d)))
dataset = pd.DataFrame(list(zip(Register_number, annex)),columns =['CAS Number', 'Annex'])

,CAS Number,Annex
0,317-34-0,Annex II
1,58-55-9,Annex II
2,75-09-2,Annex II
3,111-46-6,Annex II
4,123-99-9,Annex II
...,...,...
2046,98-01-1,ANNEX III
2047,13463-67-7,ANNEX III
2048,1317-70-0,ANNEX III
2049,1317-80-2,ANNEX III


In [19]:
CAS = pd.read_excel('Y:/delete/Chemical CAS.xlsx')
CAS['value'] = 'Low'
CAS = CAS.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [20]:
result = pd.merge(dataset, CAS, how="outer", on="CAS Number")
result.Annex.fillna(result.value, inplace=True)
del result['value']

In [21]:
result['Jurisdiction'] = 'Southeast Asia'
result['RegulatoryAgency'] = 'The Association of Southeast Asian Nations (ASEAN)'
result['TypeOfRegulation'] = 'Use'
result['Criteria'] = 'Use in cosmetics'


result['Description'] = 'Prohibited in cosmetics products'
result.loc[result['Annex'] == 'ANNEX III', 'Description'] = 'Restricted in cosmetics products'
result.loc[result['Annex'] == 'Low', 'Description'] = 'Not prohibited in cosmetics products'


result['HazardClassification'] = 'High'
result.loc[result['Annex'] == 'ANNEX III', 'HazardClassification'] = 'Moderate'
result.loc[result['Annex'] == 'Low', 'HazardClassification'] = 'Low'


result['CauseOfClassification'] = 'ASEAN Cosmetic Directive Annex II Part 1'
result.loc[result['Annex'] == 'ANNEX III', 'CauseOfClassification'] = 'ASEAN Cosmetic Directive Annex III Part 1'
result.loc[result['Annex'] == 'Low', 'CauseOfClassification'] = 'Not listed in ASEAN Cosmetic Directive Annex II Part 1 or Annex III Part 1'


result['SourceName'] = 'International Agency for Research on Cancer'
result['SourceURL'] = 'https://monographs.iarc.who.int/list-of-classifications'

result = result.drop(['Annex'], axis=1)
result.rename(columns = {'CASRN':'CAS Number'}, inplace = True)

result.head(2)

,CAS Number,Jurisdiction,RegulatoryAgency,TypeOfRegulation,Criteria,Description,HazardClassification,CauseOfClassification,SourceName,SourceURL
0,317-34-0,Southeast Asia,The Association of Southeast Asian Nations (AS...,Use,Use in cosmetics,Prohibited in cosmetics products,High,ASEAN Cosmetic Directive Annex II Part 1,International Agency for Research on Cancer,https://monographs.iarc.who.int/list-of-classi...
1,58-55-9,Southeast Asia,The Association of Southeast Asian Nations (AS...,Use,Use in cosmetics,Prohibited in cosmetics products,High,ASEAN Cosmetic Directive Annex II Part 1,International Agency for Research on Cancer,https://monographs.iarc.who.int/list-of-classi...


In [ ]:
result.to_csv('Y:/delete/Reugulatory Assessment.csv',index=False,mode='a',header=False)

In [22]:
end_time = datetime.now()
print('The total time taken is', (end_time - start_time))

The total time taken is 0:09:57.412473
